# Single Images Tutorial

This tutorial shows how to convert microscopy images into standalone OME-Zarr datasets
(without an HCS plate structure) using `ome-zarr-converters-tools`.

Use this approach when your data doesn't follow a plate layout -- for example,
individual tissue scans, brain slices, or any dataset where a well-plate hierarchy doesn't apply.

The input to the library is a **pandas DataFrame** describing your tiles.
You can build this DataFrame from any source (CSV, database, custom parsing, etc.).

**Dataset**: We reuse images from the [cardiomyocyte differentiation dataset](https://zenodo.org/records/8287221):
- 1 image ("cardiomyocyte_scan")
- 2 fields of view (FOVs)
- 2 Z-slices per FOV
- 1 channel (DAPI)

## Step 1: Prepare the tile DataFrame

The DataFrame must contain **one row per tile**. For single images, the columns are:

### Tile position and size columns

| Column | Description |
|--------|-------------|
| `file_path` | Path to the raw image file (relative to the `resource` directory, or absolute) |
| `fov_name` | Field-of-view identifier (tiles with the same `fov_name` are stitched together) |
| `start_x`, `start_y` | XY position of this tile (in the coordinate system specified by `AcquisitionDetails`) |
| `start_z` | Z position (slice index or physical position) |
| `start_c` | Channel index (0-based) |
| `start_t` | Time-point index |
| `length_x`, `length_y` | Tile dimensions in pixels |
| `length_z` | Number of Z slices in this tile (usually 1) |
| `length_c` | Number of channels in this tile (usually 1) |
| `length_t` | Number of time points in this tile (usually 1) |

### Single image column

| Column | Description |
|--------|-------------|
| `image_path` | Name of the output OME-Zarr dataset (e.g., `"brain_scan"` becomes `brain_scan.zarr`) |

### 1.1 Load the metadata

Let's load the example CSV and inspect the DataFrame.

In [ ]:
import warnings

import pandas as pd

# Suppress warnings for cleaner documentation output.
# Do not use this in production code.
warnings.filterwarnings("ignore")

tiles_table = pd.read_csv("../examples/single_acquisitions/tiles.csv")
print("Columns:", list(tiles_table.columns))
tiles_table

Note the `image_path` column -- this defines the output dataset name.
There are no `row`/`column` columns since we're not using a plate layout.

Define the acquisition details.

In [ ]:
from ome_zarr_converters_tools import AcquisitionDetails, ChannelInfo

acq = AcquisitionDetails(
    channels=[ChannelInfo(channel_label="DAPI", wavelength_id="405")],
    pixelsize=0.65,
    z_spacing=5.0,
    t_spacing=1.0,
    axes=["t", "c", "z", "y", "x"],
    start_x_coo="world",
    start_y_coo="world",
    start_z_coo="pixel",
    start_t_coo="pixel",
)
acq

## Step 2: Parse tiles from the DataFrame

Use `single_images_from_dataframe()` instead of `hcs_images_from_dataframe()`.
This creates `Tile` objects with a `SingleImage` collection type.

In [ ]:
from ome_zarr_converters_tools.core import single_images_from_dataframe

tiles = single_images_from_dataframe(
    tiles_table=tiles_table,
    acquisition_details=acq,
)

print(f"Number of tiles: {len(tiles)}")
print(f"Collection type: {type(tiles[0].collection).__name__}")
print(f"Image path: {tiles[0].collection.image_path}")
tiles[0]

Notice the collection type is `SingleImage` instead of `ImageInPlate`. The `image_path` value (`"cardiomyocyte_scan"`) will become the output Zarr name: `cardiomyocyte_scan.zarr`.

## Step 3: Aggregate tiles into TiledImages

Same pipeline as for HCS plates. The `resource` parameter points to the directory containing the image files.

In [ ]:
from ome_zarr_converters_tools import ConverterOptions, tiles_aggregation_pipeline

data_dir = "../examples/hcs_plate/data"
opts = ConverterOptions()

tiled_images = tiles_aggregation_pipeline(
    tiles=tiles,
    converter_options=opts,
    resource=data_dir,
)

print(f"Number of TiledImages: {len(tiled_images)}")
for ti in tiled_images:
    print(f"  Path: {ti.path}, regions: {len(ti.regions)}, FOVs: {len(ti.group_by_fov())}")

## Step 4: Register and write OME-Zarr

For single images, there is no plate structure to set up. We go directly to the registration
pipeline and writing step.

In [ ]:
import tempfile

from ome_zarr_converters_tools.models import (
    AlignmentCorrections,
    OverwriteMode,
    TilingMode,
    WriterMode,
)
from ome_zarr_converters_tools.pipelines import (
    build_default_registration_pipeline,
    tiled_image_creation_pipeline,
)

pipeline = build_default_registration_pipeline(
    alignment_corrections=AlignmentCorrections(),
    tiling_mode=TilingMode.AUTO,
)

zarr_dir = tempfile.mkdtemp(prefix="tutorial_images_")

for tiled_image in tiled_images:
    zarr_url = f"{zarr_dir}/{tiled_image.path}"
    omezarr = tiled_image_creation_pipeline(
        zarr_url=zarr_url,
        tiled_image=tiled_image,
        registration_pipeline=pipeline,
        converter_options=opts,
        writer_mode=WriterMode.BY_FOV,
        overwrite_mode=OverwriteMode.OVERWRITE,
        resource=data_dir,
    )
    print(f"Written: {zarr_url}")

## Step 5: Verify the result

In [ ]:
img = omezarr.get_image()
data = img.get_array()

print(f"Shape (t, c, z, y, x): {data.shape}")
print(f"Channels: {img.channel_labels}")
print(f"Tables: {omezarr.list_tables()}")

## Cleanup

In [ ]:
import shutil

shutil.rmtree(zarr_dir, ignore_errors=True)
print("Cleaned up tutorial output.")